In [1]:
import os
import re
import pandas as pd

In [2]:
results_folder = [os.pardir,"Benchmark results","halloc_opts"]
# results_folder = [os.pardir,"Benchmark results"]
print(os.path.join(*results_folder))


../Benchmark results/halloc_opts


In [3]:
parsable_keys = {
    "HALLOC_FRACTION": "hf-",
    "BUSY_FRACTION": "bu-",
    "ROOMY_FRACTION": "ro-",
    "SPARSE_FRACTION": "sp-",
    "MAX_BLOCK_SZ": "mb-",
    "MAX_NSIZES": "ns-",
    "MAX_NCHUNK_IDS": "nc-",
    "BLOCK_STEP": "bst-",
    "NUNITS": "nu-",
    "MIN_BLOCK_SZ": "mibs-",
    "MAX_BLOCK_SZ": "mabs-",
}

In [4]:
default_params = {
    "HALLOC_FRACTION": 0.25,
    "BUSY_FRACTION": 0.835,
    "ROOMY_FRACTION": 0.6,
    "SPARSE_FRACTION": 0.012,
    "MAX_BLOCK_SZ": 3072,
    "MAX_NSIZES": 16,
    "MAX_NCHUNK_IDS": 8,
    "BLOCK_STEP": 16,
    "NUNITS": 8,
    "MIN_BLOCK_SZ": 16,
    "MAX_BLOCK_SZ": 3072,
}

In [5]:
expr = "total\s+images\/sec:\s+([0-9]*\.?[0-9]+)"
expr = r'total\s+images\/sec:\s+(\d+\.*\d*)'
r = re.compile(expr)

In [6]:
values_dict = {}

In [7]:
cols = ["ALLOCATOR", "NETWORK", "IMAGES_SEC"] + list(parsable_keys.keys())
df = pd.DataFrame(columns=cols)
print(df)

Empty DataFrame
Columns: [ALLOCATOR, NETWORK, IMAGES_SEC, HALLOC_FRACTION, BUSY_FRACTION, ROOMY_FRACTION, SPARSE_FRACTION, MAX_BLOCK_SZ, MAX_NSIZES, MAX_NCHUNK_IDS, BLOCK_STEP, NUNITS, MIN_BLOCK_SZ]
Index: []


In [8]:
for file in os.listdir(os.path.join(*results_folder)):
    #do something
	print(file)
	file_name = os.path.splitext(file)
	if file_name[-1] == ".txt":
		params = default_params
		print(file_name[0].split('_'))
		
		file_split = file_name[0].split('_')
		params["ALLOCATOR"] = file_split.pop(0)
		params["NETWORK"] = file_split.pop(0)
		for part in file_split:
			for key, value in parsable_keys.items():
				if value in part:
					print(key, part.split('-')[-1])
					params[key]=float(part.split('-')[-1])
		print(params)
		
		matches = [re.findall(expr,line) for line in open(os.path.join(*results_folder, file))]
		matches = [float(x[0]) for x in matches if len(x)]
		print(matches)
		
		if len(matches):
			for match in matches:
				params["IMAGES_SEC"] = match
				df = df.append(params, ignore_index=True)
		else:
			params["IMAGES_SEC"] = float('NaN')
			df = df.append(params, ignore_index=True)
df

results.csv


,ALLOCATOR,NETWORK,IMAGES_SEC,HALLOC_FRACTION,BUSY_FRACTION,ROOMY_FRACTION,SPARSE_FRACTION,MAX_BLOCK_SZ,MAX_NSIZES,MAX_NCHUNK_IDS,BLOCK_STEP,NUNITS,MIN_BLOCK_SZ


In [9]:
df.to_csv(os.path.join(*results_folder,'results.csv'))